In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import hoomd
import gsd.hoomd
import matplotlib.pyplot as plt

import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [50]:
#cant get the sdf file or mol file to work with N-S-N bonds, it works from smiles string though...
#either need to get smiles string directly from mol2 file or figure out how to get the 
#smiles string from the smi file into a usable format. 
#it worked to run openbabel and get the smile string from the smi file and then just 
#delete the brackets around the N's.

#molecule = Molecule.from_file("C5.sdf",file_format = "sdf")

molecule = Molecule.from_smiles("C1([C]2[C](S[CH][CH]2)[C]2[C]1[CH][C](S2)[C]1[CH][N][CH][C]2[C]1NSN2)(CCCCCCCCCCCCCCCC)CCCCCCCCCCCCCCCC")
molecule_graph = esp.Graph(molecule)
#either generate smiles string from the built fragments or get our fragments into a Molecule supported file format

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

/Users/madilyn/miniconda3/envs/espaloma/bin/wrapped_progs/antechamber: Fatal Error!
Cannot properly run "/Users/madilyn/miniconda3/envs/espaloma/bin/sqm -O -i sqm.in -o sqm.out".


KeyboardInterrupt: 

In [3]:
# get a parmed structure from openmm 
import parmed as pmd
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()

structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)

In [4]:
# step 1: how to get bond parameters

bond_parameters = {}

for bond in structure.bonds:
    bond_parameters[(bond.atom1.name, bond.atom2.name)] = {"k":[bond.type.k] , "l0":[bond.type.req]}
    
# step 2: get angle parameters

angle_parameters = {}

for angle in structure.angles:
    angle_parameters[(angle.atom1.name, angle.atom2.name,angle.atom3.name)] = {"k":[angle.type.k], 
                                                                               "t0":[angle.type.theteq]}
    
# step 3: dihedral parameters

dihedral_parameters = {}

for dihedral in structure.dihedrals:
    dihedral_parameters[(dihedral.atom1.name, dihedral.atom2.name,dihedral.atom3.name,
                         dihedral.atom4.name)] = {"periodicity":[dihedral.type.per],
                                                  "k":[dihedral.type.phi_k],"phase":[dihedral.type.phase]}
    
# step 4: non-bonding parameters

nonbonded_parameters = {}

for nonbonded in structure.adjusts:
    nonbonded_parameters[(nonbonded.atom1.name,nonbonded.atom2.name)] = {"sigma":[nonbonded.type.rmin],
                                                    "epsilon":[nonbonded.type.epsilon],
                                                    "charge":[nonbonded.type.chgscale]}

In [5]:
particle_types = []
particle_type_dict = dict()

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge, sigma, epsilon) not in particle_types: #this is adding the charge,sigma, and epsilon to the particle_types, but only once.
        particle_types.append((charge, sigma, epsilon))
        #print((charge, sigma, epsilon) )
    particle_type_dict[i] = particle_types.index((charge, sigma, epsilon)) #adding only the index of the pair_parms, this gives particle types.
print(particle_type_dict)

#particle_types gives the sigma, charge, and epsilon associated with each unique particle, 
#particle_type_dict only gives the index associated with each unique particle type

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 18, 21: 19, 22: 19, 23: 20, 24: 20, 25: 20}


In [6]:
#graph check
#want to check if the opemm and parmed graphs are the same so we can use types from parmed 
import networkx  as nx
Gopenmm = nx.Graph()
Gparmed = nx.Graph()
#openmm:
for i in range(bond_forces.getNumBonds()):
    Gopenmm.add_edge(bond_forces.getBondParameters(index=i)[0],bond_forces.getBondParameters(index=i)[1])
#parmed
for b in structure.bonds:
    Gparmed.add_edge(b.atom1.idx,b.atom2.idx)
type_map = {}
#nx.rooted_tree_isomorphism
#in here we still need to check that one known index on one corresponds to the same index on the other....
tree_openmm = nx.bfs_tree(Gopenmm,0)
tree_parmed = nx.bfs_tree(Gparmed,0)
if nx.is_isomorphic(Gopenmm,Gparmed):
#if nx.isomorphism.tree_isomorphism(tree_openmm,tree_parmed):  <- want this work
    for b in structure.bonds:
        type_map[b.atom1.idx] = b.atom1.atom_type.__str__()
        type_map[b.atom2.idx] = b.atom2.atom_type.__str__()
print(type_map)

{1: 'C2', 0: 'C1', 5: 'C2', 2: 'C2', 3: 'S1', 4: 'C2', 6: 'C2', 10: 'S1', 7: 'C2', 8: 'C2', 9: 'C2', 16: 'C1', 11: 'H1', 12: 'H1', 13: 'H2', 14: 'H3', 15: 'H3', 17: 'C1', 18: 'C1', 19: 'H1', 20: 'H1', 21: 'H1', 22: 'H1', 23: 'H1', 24: 'H1', 25: 'H1'}


In [7]:
# Rename the particle types so that they match the xml file
# This is needed when we aren't using SMARTS matching with Foyer.

c5 = mb.load("C5.mol2")

for index in type_map:
    c5[index].name = type_map[index]

In [8]:
bond_types = []
bond_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    #print(bond_parms)
    #atoms = {(bond.atom1.name,bond.atom2.name):[]} #gives you the atom type name and index
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    #if (type_map[bond_parms[0]],type_map[bond_parms[1]],k,l0) not in bond_dict:
    #    bond_types.append((type_map[bond_parms[0]],type_map[bond_parms[1]],k,l0))
    bond_dict[type_map[bond_parms[0]],type_map[bond_parms[1]]] = {'k':k,'l0':l0}
print(bond_dict)

{('C1', 'C2'): {'k': 133294.27795350857, 'l0': 0.15228319801780385}, ('C1', 'H1'): {'k': 367143.4369119569, 'l0': 0.11571172936022506}, ('C2', 'C2'): {'k': 79255.34916790333, 'l0': 0.1521817355789801}, ('C2', 'S1'): {'k': 21114.006445429724, 'l0': 0.18958480007468242}, ('S1', 'C2'): {'k': 28694.966463816825, 'l0': 0.18876555585650523}, ('C2', 'H2'): {'k': 315697.8888381377, 'l0': 0.10270294393487327}, ('C2', 'H3'): {'k': 195188.3176604623, 'l0': 0.10494616970245124}, ('C2', 'C1'): {'k': 174204.69976096076, 'l0': 0.1451684252139238}, ('C1', 'C1'): {'k': 256858.34515821582, 'l0': 0.15905723517317444}}


In [9]:
angle_types = []
angle_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit  
#    if (type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]],k,t0) not in angle_types:
#        angle_types.append((type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]],k,t0))
    angle_dict[type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]]] = {'k':k,'t0':t0}
print(angle_dict)

{('C1', 'C2', 'C2'): {'k': 308.70868383326155, 't0': 1.894422173500061}, ('C2', 'C1', 'C2'): {'k': 632.8632819300277, 't0': 1.8301576375961304}, ('C2', 'C1', 'H1'): {'k': 343.88739479425425, 't0': 1.9283297061920166}, ('C2', 'C2', 'S1'): {'k': 270.93150190046146, 't0': 1.8118257522583008}, ('C2', 'C2', 'C2'): {'k': 503.57289986390884, 't0': 1.8925292491912842}, ('C2', 'C2', 'H3'): {'k': 372.1358782760925, 't0': 2.002474308013916}, ('C2', 'S1', 'C2'): {'k': 391.468035960259, 't0': 1.534282922744751}, ('S1', 'C2', 'C2'): {'k': 313.25530382004433, 't0': 2.0114521980285645}, ('S1', 'C2', 'H2'): {'k': 320.7375371737616, 't0': 2.1473066806793213}, ('C2', 'C2', 'H2'): {'k': 388.3106930285571, 't0': 2.039250373840332}, ('C2', 'C2', 'C1'): {'k': 304.77293026452503, 't0': 1.9249714612960815}, ('C2', 'C1', 'C1'): {'k': 341.8755731076143, 't0': 2.0228121280670166}, ('S1', 'C2', 'C1'): {'k': 244.62416441779027, 't0': 1.8533281087875366}, ('H1', 'C1', 'H1'): {'k': 448.81099853196895, 't0': 2.0283789

In [10]:
dihedral_types = []
dihedral_dict = {}

for i in range(torsion_forces.getNumTorsions()):
    if i%6==0:
        periodicity=[]
        phase = []
        k = []
    dihedral_parms = torsion_forces.getTorsionParameters(index=i)
    periodicity.append(dihedral_parms[4])  
    phase.append( dihedral_parms[5]/dihedral_parms[5].unit)
    k.append(dihedral_parms[6]/dihedral_parms[6].unit)
#    if (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],type_map[dihedral_parms[3]],periodicity,k,phase) not in dihedral_types:
#        dihedral_types.append((type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
#        type_map[dihedral_parms[3]],periodicity,k,phase))
    dt = (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
                  type_map[dihedral_parms[3]])
   

    if periodicity[-1]==6:
        #print(dt,periodicity,phase)
        dihedral_dict[dt] = {'periodicity':periodicity,'k':k,'phase':phase}
print(dihedral_dict)

{('C1', 'C2', 'C2', 'S1'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [2.243226856698272, 2.6613758547276207, 0.11668677283042739, 2.013308360386957, 0.47831771660268824, 1.1670633432709854], 'phase': [3.141592653589793, 3.141592653589793, 0.0, 0.0, 0.0, 3.141592653589793]}, ('C1', 'C2', 'C2', 'C2'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [0.6032129824737422, 3.4322210829246877, 0.7715152094580535, 0.819685395010733, 0.9580340802173629, 0.855737233375642], 'phase': [0.0, 3.141592653589793, 0.0, 0.0, 3.141592653589793, 3.141592653589793]}, ('C1', 'C2', 'C2', 'H3'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [3.3635333325510675, 2.506036787068565, 0.6030907231195477, 0.6834982551862641, 1.3595337993924888, 0.21699201479231792], 'phase': [0.0, 3.141592653589793, 0.0, 3.141592653589793, 0.0, 3.141592653589793]}, ('C2', 'C1', 'C2', 'C2'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [4.0184326796041265, 3.962859690154963, 0.5619822378651492, 0.0381253570120497, 0.44654984600875863, 0.154473471431

In [11]:
nonbonded_types = []
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
    #atom_type = structure.adjusts.atom
    if (charge,sigma,epsilon) not in nonbonded_types:
        nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':charge,'sigma':sigma,'epsilon':epsilon}
print(nonbonded_dict)

{'C1': {'charge': -0.09417688461538462, 'sigma': 0.3399669508423535, 'epsilon': 0.4577296}, 'C2': {'charge': -0.12047688461538461, 'sigma': 0.3399669508423535, 'epsilon': 0.359824}, 'S1': {'charge': 0.08012311538461538, 'sigma': 0.35635948725613575, 'epsilon': 1.046}, 'H1': {'charge': 0.034956115384615384, 'sigma': 0.2649532787749369, 'epsilon': 0.06568879999999999}, 'H2': {'charge': 0.1689231153846154, 'sigma': 0.2510552587719476, 'epsilon': 0.06276}, 'H3': {'charge': 0.1549231153846154, 'sigma': 0.25996424595335105, 'epsilon': 0.06276}}


In [12]:
# Save the forcefield XML file for future use, so that we don't have to repeat the espaloma process everytime
mbuild_to_foyer_xml(
    file_name="c5.xml",
    compound=c5,
    bond_params=bond_dict,
    angle_params=angle_dict,
    dihedral_params=dihedral_dict,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_dict,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

# Save the mb.Compound with the new atom type names for future use.
c5.save("c5_typed.mol2", overwrite=True)

## Workflow once we have created an XML file, and a typed mol2 file:

Everything above this cell is something that we should only need to do once.
Then, once we have an XML file, and a typed mol2 file (i.e. one with the particles renamed correctly)
every time we want to run a new simulation, the process would look like the cells below:

In [13]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [14]:
esp_ff = foyer.Forcefield(forcefield_files="c5.xml")

In [15]:
# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
c5s = mb.load("c5_typed.mol2")
for p in c5s.particles():
    p.name = f"_{p.name}"

In [16]:
box = mb.fill_box(compound=c5s, n_compounds=5, box=[1,1,1])
box_pmd = esp_ff.apply(box)

RuntimeError: PACKMOL failed. See 'log.txt'

In [ ]:
snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)

In [ ]:
#setting cpu and simulation 
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

In [ ]:
#setting the integrator
kt = 4
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt=0.01)
nvt = hoomd.md.methods.NVT(kT=kt,filter=all, tau=0.1)
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_c5s.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)